In [1]:
import imagehash
from PIL import Image

In [2]:
import pandas as pd

Считаем хеши всех фотографий в трех базах

## База 3

In [ ]:
df = pd.read_csv('data_prepped/base_3.csv', index_col=0)
df.head()

In [ ]:
df.columns

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
def download(url_1, url_2):
    pic_1_content = requests.get(url_1).content
    pic_2_content = requests.get(url_2).content
    
    with open ('pic_1.jpg', 'wb') as pic_1_file:
        pic_1_file.write(pic_1_content)
    with open ('pic_2.jpg', 'wb') as pic_2_file:
        pic_2_file.write(pic_2_content)

In [ ]:
def get_hashes():
    pic_1 = Image.open('pic_1.jpg')
    hash_1 = imagehash.dhash(pic_1)
    pic_2 = Image.open('pic_2.jpg')
    hash_2 = imagehash.dhash(pic_2)
    return str(hash_1), str(hash_2)

In [ ]:
def calculate(row):
    url_1 = row['Pic_url_1']
    url_2 = row['Pic_url_2']
    
    try:
        download(url_1, url_2)
        return get_hashes() 
    
    except:
        raise
#         return 'error', 'error'

In [ ]:
from tqdm import tqdm
tqdm.pandas()

In [ ]:
sample = df.head()
sample['calc_res'] = sample.progress_apply(calculate, axis=1)
sample

In [ ]:
df['calc_res'] = df.progress_apply(calculate, axis=1)

In [ ]:
df.to_csv('base_3_hashes.csv')

## База 2

In [ ]:
df = pd.read_csv('data_prepped/base_2.csv', index_col=0)
df.head()

In [ ]:
df['calc_res'] = df.progress_apply(calculate, axis=1)

In [ ]:
df.to_csv('base_2_hashes.csv')

## Обработка

База 2

In [ ]:
df2 = pd.read_csv('base_2_hashes.csv', index_col=0)
df2.head()

In [ ]:
df2['base'] = 2
df2 = df2[['base', 'ID', 'Pic_url_1', 'Pic_url_2', 'calc_res']]
df2.head()

In [ ]:
import ast

In [ ]:
df2['calc_res'] = df2['calc_res'].apply(ast.literal_eval)
df2['hash_1'] = df2['calc_res'].apply(lambda x: x[0])
df2['hash_2'] = df2['calc_res'].apply(lambda x: x[1])
df2 = df2.drop('calc_res', axis=1)
df2.head()

6 случаев - ошибка скачивания (битые ссылки)

In [ ]:
df2[df2['hash_1'] == 'error']

In [ ]:
df2[df2['hash_1'] == 'error'].to_csv('base_2_download_error.csv')

In [ ]:
df2 = df2[df2['hash_1'] != 'error']
df2.shape

База 3

In [ ]:
df3 = pd.read_csv('base_3_hashes.csv', index_col=0)
df3.head()

In [ ]:
df3['base'] = 3
df3 = df3[['base', 'ID', 'Pic_url_1', 'Pic_url_2', 'calc_res']]

df3['calc_res'] = df3['calc_res'].apply(ast.literal_eval)
df3['hash_1'] = df3['calc_res'].apply(lambda x: x[0])
df3['hash_2'] = df3['calc_res'].apply(lambda x: x[1])
df3 = df3.drop('calc_res', axis=1)

df3.head()

Ошибка скачивания

In [ ]:
(df3['hash_1'] == 'error').sum()

In [ ]:
df3[df3['hash_1'] == 'error']

In [ ]:
df3[df3['hash_1'] == 'error'].to_csv('base_3_download_error.csv')

In [ ]:
df3 = df3[df3['hash_1'] != 'error']
df3.shape

In [ ]:
df = df2.append(df3)
df.head()

In [ ]:
df.shape

In [ ]:
df.to_csv('base_2_3_hashes.csv')

## База 1 (основная база)

In [ ]:
df = pd.read_csv('data_prepped/base_1.csv', index_col=0)
df.head()

In [ ]:
(df['Pic_url_2'] == '0.0').sum()

In [ ]:
df.shape

In [ ]:
def download(url):
    content = requests.get(url).content
    with open ('pic.jpg', 'wb') as pic_file:
        pic_file.write(content)

In [ ]:
def get_hash():
    try:
        pic = Image.open('pic.jpg')
        hash_value = imagehash.dhash(pic)
    except:
        hash_value = 'hash_error'
    return str(hash_value)

In [ ]:
def calculate(row):
    url_1 = row['Pic_url_1']
    url_2 = row['Pic_url_2']
    
    try:
        download(url_1)
        res_1 = get_hash()
    except:
        res_1 = 'dwnld_error'
    
    if url_2 == '0.0':
        res_2 = 'no_url'
    else:
        try:
            download(url_2)
            res_2 = get_hash()
        except:
            res_2 = 'dwnld_error'
    
    return res_1, res_2

In [ ]:
sample = df.head()
sample['calc_res'] = sample.progress_apply(calculate, axis=1)
sample

In [ ]:
df['calc_res'] = df.progress_apply(calculate, axis=1)

In [ ]:
df.to_csv('base_1_hashes.csv')

# Объединение в один датафрейм

In [18]:
df = pd.read_csv('hash_values/base_1_hashes.csv', index_col=0)
df.head()

,ID,Pic_url_1,Pic_url_2,calc_res
0,0,https://forum.vgd.ru/file.php?fid=5476&key=0,0.0,"('806969715521b059', 'no_url')"
2,2,https://forum.vgd.ru/file.php?fid=5492&key=0,0.0,"('f4d8cc0ca543d6b6', 'no_url')"
3,3,https://forum.vgd.ru/file.php?fid=20890&key=19...,0.0,"('f1732b4d73511723', 'no_url')"
4,4,https://forum.vgd.ru/file.php?fid=20894&key=16...,0.0,"('b292ced8b3f2f2f2', 'no_url')"
5,5,https://forum.vgd.ru/file.php?fid=20895&key=90...,0.0,"('b8186ca4ae8a989a', 'no_url')"


In [4]:
import ast

In [19]:
df['base'] = 1
df = df[['base', 'ID', 'Pic_url_1', 'Pic_url_2', 'calc_res']]

df['calc_res'] = df['calc_res'].apply(ast.literal_eval)
df['hash_1'] = df['calc_res'].apply(lambda x: x[0])
df['hash_2'] = df['calc_res'].apply(lambda x: x[1])
df = df.drop('calc_res', axis=1)

df.head()

,base,ID,Pic_url_1,Pic_url_2,hash_1,hash_2
0,1,0,https://forum.vgd.ru/file.php?fid=5476&key=0,0.0,806969715521b059,no_url
2,1,2,https://forum.vgd.ru/file.php?fid=5492&key=0,0.0,f4d8cc0ca543d6b6,no_url
3,1,3,https://forum.vgd.ru/file.php?fid=20890&key=19...,0.0,f1732b4d73511723,no_url
4,1,4,https://forum.vgd.ru/file.php?fid=20894&key=16...,0.0,b292ced8b3f2f2f2,no_url
5,1,5,https://forum.vgd.ru/file.php?fid=20895&key=90...,0.0,b8186ca4ae8a989a,no_url


Проверки

In [ ]:
(df['hash_2'] == 'no_url').sum()

In [ ]:
(df['hash_1'] == 'dwnld_error').sum()

In [ ]:
df[(df['hash_1'] == 'dwnld_error') | (df['hash_2'] == 'dwnld_error')].shape

In [ ]:
df[(df['hash_1'] == 'hash_error') | (df['hash_2'] == 'hash_error')].shape

In [ ]:
df[((df['hash_1'] == 'dwnld_error') & ((df['hash_2'] == 'dwnld_error') | (df['hash_2'] == 'no_url')))
  | ((df['hash_1'] == 'hash_error') & (df['hash_2'] == 'hash_error'))].shape

In [ ]:
df[((df['hash_1'] == 'dwnld_error') & ((df['hash_2'] == 'dwnld_error') | (df['hash_2'] == 'no_url')))
  | ((df['hash_1'] == 'hash_error') & (df['hash_2'] == 'hash_error'))].to_csv('base_1_download_error.csv')

In [20]:
df = df[~(((df['hash_1'] == 'dwnld_error') & ((df['hash_2'] == 'dwnld_error') | (df['hash_2'] == 'no_url')))
  | ((df['hash_1'] == 'hash_error') & (df['hash_2'] == 'hash_error')))]
df.shape

(7147, 6)

Добавляем вторую и третью базу

In [21]:
df = df.append(pd.read_csv('hash_values/base_2_3_hashes.csv', index_col=0))
df.head()

C:\Users\811156\AppData\Local\Temp\ipykernel_17016\2197507994.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.read_csv('hash_values/base_2_3_hashes.csv', index_col=0))


,base,ID,Pic_url_1,Pic_url_2,hash_1,hash_2
0,1,0,https://forum.vgd.ru/file.php?fid=5476&key=0,0.0,806969715521b059,no_url
2,1,2,https://forum.vgd.ru/file.php?fid=5492&key=0,0.0,f4d8cc0ca543d6b6,no_url
3,1,3,https://forum.vgd.ru/file.php?fid=20890&key=19...,0.0,f1732b4d73511723,no_url
4,1,4,https://forum.vgd.ru/file.php?fid=20894&key=16...,0.0,b292ced8b3f2f2f2,no_url
5,1,5,https://forum.vgd.ru/file.php?fid=20895&key=90...,0.0,b8186ca4ae8a989a,no_url


In [22]:
df.shape

(17082, 6)

In [23]:
df.isna().sum()

base         0
ID           0
Pic_url_1    0
Pic_url_2    0
hash_1       0
hash_2       0
dtype: int64

In [24]:
df['base'].value_counts()

1    7147
2    4970
3    4965
Name: base, dtype: int64

Разделяем лицевую и обратную сторону открытки на две разные строки

In [25]:
df_pic_1 = df[['base', 'ID', 'Pic_url_1', 'hash_1']]
df_pic_1['Pic'] = 1
df_pic_1 = df_pic_1.rename(columns={'Pic_url_1': 'Pic_url', 'hash_1': 'hash'})
df_pic_1 = df_pic_1[['base', 'ID', 'Pic', 'Pic_url', 'hash']]
df_pic_1.head()

C:\Users\811156\AppData\Local\Temp\ipykernel_17016\376422897.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pic_1['Pic'] = 1


,base,ID,Pic,Pic_url,hash
0,1,0,1,https://forum.vgd.ru/file.php?fid=5476&key=0,806969715521b059
2,1,2,1,https://forum.vgd.ru/file.php?fid=5492&key=0,f4d8cc0ca543d6b6
3,1,3,1,https://forum.vgd.ru/file.php?fid=20890&key=19...,f1732b4d73511723
4,1,4,1,https://forum.vgd.ru/file.php?fid=20894&key=16...,b292ced8b3f2f2f2
5,1,5,1,https://forum.vgd.ru/file.php?fid=20895&key=90...,b8186ca4ae8a989a


In [26]:
df_pic_2 = df[['base', 'ID', 'Pic_url_2', 'hash_2']]
df_pic_2['Pic'] = 2
df_pic_2 = df_pic_2.rename(columns={'Pic_url_2': 'Pic_url', 'hash_2': 'hash'})
df_pic_2 = df_pic_2[['base', 'ID', 'Pic', 'Pic_url', 'hash']]
df_pic_2.head()

C:\Users\811156\AppData\Local\Temp\ipykernel_17016\799477740.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pic_2['Pic'] = 2


,base,ID,Pic,Pic_url,hash
0,1,0,2,0.0,no_url
2,1,2,2,0.0,no_url
3,1,3,2,0.0,no_url
4,1,4,2,0.0,no_url
5,1,5,2,0.0,no_url


In [27]:
df = df_pic_1.append(df_pic_2)
df.head()

C:\Users\811156\AppData\Local\Temp\ipykernel_17016\2336499448.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df_pic_1.append(df_pic_2)


,base,ID,Pic,Pic_url,hash
0,1,0,1,https://forum.vgd.ru/file.php?fid=5476&key=0,806969715521b059
2,1,2,1,https://forum.vgd.ru/file.php?fid=5492&key=0,f4d8cc0ca543d6b6
3,1,3,1,https://forum.vgd.ru/file.php?fid=20890&key=19...,f1732b4d73511723
4,1,4,1,https://forum.vgd.ru/file.php?fid=20894&key=16...,b292ced8b3f2f2f2
5,1,5,1,https://forum.vgd.ru/file.php?fid=20895&key=90...,b8186ca4ae8a989a


In [28]:
df.shape

(34164, 5)

In [29]:
df = df[~df['hash'].isin(['no_url', 'dwnld_error', 'hash_error', 'error'])]
df.shape

(33932, 5)

In [30]:
df['base'].value_counts()

1    14074
3     9930
2     9928
Name: base, dtype: int64

In [31]:
df.to_csv('hash_values/hash_database_full_new.csv')

## Поиск дублирования

In [15]:
df = pd.read_csv('hash_values/hash_database_full.csv', index_col=0)
df = df.reset_index().drop('index', axis=1)
df.head()

,base,ID,Pic,Pic_url,hash
0,1,0,1,https://forum.vgd.ru/file.php?fid=5476&key=0,806969715521b059
1,1,2,1,https://forum.vgd.ru/file.php?fid=5492&key=0,f4d8cc0ca543d6b6
2,1,3,1,https://forum.vgd.ru/file.php?fid=20890&key=19...,f1732b4d73511723
3,1,4,1,https://forum.vgd.ru/file.php?fid=20894&key=16...,b292ced8b3f2f2f2
4,1,5,1,https://forum.vgd.ru/file.php?fid=20895&key=90...,b8186ca4ae8a989a


In [4]:
df.shape

(44884, 5)

In [5]:
df['base'].value_counts()

3    20882
1    14074
2     9928
Name: base, dtype: int64

In [27]:
df['hash'].nunique()

44029

In [36]:
44029 * (36 + 36 ** 2) / 1000

58646.628

In [34]:
44029 * (44029 - 1) // 2 / 1000

969254.406

Посчитаем, сколько раз встречается каждый хеш

In [16]:
df['hash_count'] = df.groupby('hash')['ID'].transform('count')
df.head()

,base,ID,Pic,Pic_url,hash,hash_count
0,1,0,1,https://forum.vgd.ru/file.php?fid=5476&key=0,806969715521b059,1
1,1,2,1,https://forum.vgd.ru/file.php?fid=5492&key=0,f4d8cc0ca543d6b6,1
2,1,3,1,https://forum.vgd.ru/file.php?fid=20890&key=19...,f1732b4d73511723,1
3,1,4,1,https://forum.vgd.ru/file.php?fid=20894&key=16...,b292ced8b3f2f2f2,1
4,1,5,1,https://forum.vgd.ru/file.php?fid=20895&key=90...,b8186ca4ae8a989a,1


Максимальный размер кластера фотографий с полностью совпадающими хешами

In [13]:
df['hash_count'].max()

15

In [17]:
df[df['hash_count'] == df['hash_count'].max()]

,base,ID,Pic,Pic_url,hash,hash_count
30686,3,6689,2,https://storage.yandexcloud.net/postcards/Ti4 ...,0001000000000000,15
30693,3,6696,2,https://storage.yandexcloud.net/postcards/Ti4 ...,0001000000000000,15
30725,3,6728,2,https://storage.yandexcloud.net/postcards/Ti4 ...,0001000000000000,15
30729,3,6732,2,https://storage.yandexcloud.net/postcards/Ti4 ...,0001000000000000,15
30734,3,6737,2,https://storage.yandexcloud.net/postcards/Ti4 ...,0001000000000000,15
30742,3,6745,2,https://storage.yandexcloud.net/postcards/Ti4 ...,0001000000000000,15
30745,3,6748,2,https://storage.yandexcloud.net/postcards/Ti4 ...,0001000000000000,15
30749,3,6752,2,https://storage.yandexcloud.net/postcards/Ti4 ...,0001000000000000,15
30750,3,6753,2,https://storage.yandexcloud.net/postcards/Ti4 ...,0001000000000000,15
30753,3,6756,2,https://storage.yandexcloud.net/postcards/Ti4 ...,0001000000000000,15


In [23]:
df.iloc[30749]['Pic_url']

'https://storage.yandexcloud.net/postcards/Ti4 (180).JPG'

Всего таких кластеров 787, в них суммарно входит 1642 фотографии

In [25]:
df[df['hash_count'] > 1]['hash'].nunique()

787

In [26]:
df[df['hash_count'] > 1].shape

(1642, 6)

Для каждого хеша рассчитаем все хеши, расстояние до которых от данного составляет 1 или 2

In [37]:
unique_hashes = list(df['hash'].unique())
len(unique_hashes)

44029

In [41]:
alphabet = 'abcdefghijklmnopqrstuwvxyz0123456789'
len(alphabet)

36

In [42]:
def get_hashes_at_dist_1(input_hash):
    res = []
    for i in range(16):
        res += [input_hash[:i] + s + input_hash[i + 1:] for s in alphabet]
    return res

In [46]:
res = get_hashes_at_dist_1('806969715521b059')
h = imagehash.hex_to_hash('806969715521b059')
print(h)
for v in res:
    v = imagehash.hex_to_hash(v)
    if h - v != 1:
        print(v, h - v)

806969715521b059
b06969715521b059 2
d06969715521b059 2
e06969715521b059 2
f06969715521b059 3


ValueError: invalid literal for int() with base 16: 'g06969715521b059'

### Совпадающие хеши

In [ ]:
df[df['hash_count'] > 1].sort_values(by='hash')

In [ ]:
df[df['hash_count'] > 1].sort_values(by='hash').to_csv('full_hash_doubles.csv', sep=';')

### Близкие хеши

In [ ]:
df['hash'].nunique()

In [ ]:
df['hash'].unique()

Считаем расстояние между всеми парами

In [ ]:
df = pd.read_csv('hash_database_full.csv', index_col=0)
df.head()

In [ ]:
import numpy as np

In [ ]:
from tqdm import tqdm

In [ ]:
n = df.shape[0]
n

In [ ]:
hash_distances = pd.read_csv('hash_distances.csv', index_col=0)
hash_distances.head()

In [ ]:
hash_distances = hash_distances.to_numpy()

In [ ]:
for i in tqdm(range(n)):
    if i < 206:
        continue
    
    for j in range(n):
        if not np.isnan(hash_distances[i, j]):
            continue
        if i == j:
            hash_distances[i][j] = 0
            continue
            
        hash_1 = imagehash.hex_to_hash(df.iloc[i]['hash'])
        hash_2 = imagehash.hex_to_hash(df.iloc[j]['hash'])
        hash_distances[i, j] = hash_1 - hash_2
        hash_distances[j, i] = hash_1 - hash_2

In [ ]:
pd.DataFrame(hash_distances).to_csv('hash_distances.csv')

Результат - матрица n x n, где на пересечении столбцов i, j записано расстояние между хешами фотографий, индексы (!) которых в датафрейме `hash_database_full` равны i, j. Перестановка i, j местами не имеет значения  
Анализ матрицы в тетрадке `doubles_search`